<a href="https://colab.research.google.com/github/kozeljko/nlp-models/blob/master/lstm_glove300_sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Init environment

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# How I got Glove6B embeddings
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!ls
#!unzip glove.6B.zip
#!mkdir drive/MyDrive/nlp/models-pre/glove6B
#!mv *d.txt drive/MyDrive/nlp/models-pre/glove6B

In [4]:
!pip install lemmagen3 emoji
!pip install --upgrade keras
!pip install tensorflow-addons

import sys
sys.path.append('drive/MyDrive/nlp/nlp-offensive-language/src')


from preprocessing import preprocess



     |████████████████████████████████| 12.4MB 243kB/s 
     |████████████████████████████████| 133kB 39.6MB/s 
     |████████████████████████████████| 194kB 41.3MB/s 
Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)
     |████████████████████████████████| 706kB 4.1MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [146]:
DATASETS_DIR = "drive/MyDrive/nlp/nlp-offensive-language/datasets/"

GLOVE_DIR = "drive/MyDrive/nlp/models-pre/glove6B/"

MAX_SEQUENCE_LENGTH = 100

In [147]:
import os, csv
import numpy as np
import pandas as pd
from preprocessing import *

#filename = "english/fox_news/dataset.csv"
#filename = "english/gab_and_reddit/dataset.csv"
#filename = "english/deep_offense/dataset.csv"
#filename = "english/trac_2/dataset.csv"
filename = "english/wiki_detox/dataset_aggression.csv"
#filename = "english/wiki_detox/dataset_attack.csv"
#filename = "english/wiki_detox/dataset_toxicity.csv"
csv_read = csv.reader(open(os.path.join(DATASETS_DIR, filename), encoding="utf8"), delimiter=",")

texts = []
labels = []
for line in csv_read:
  if line[0] == "id":
    continue

  text = line[1]
  text = text.replace("NEWLINE_TOKEN", "")
  text = preprocess(text, [PP_LOWERCASE, PP_REMOVE_USERNAME_HANDLES, PP_REMOVE_URLS, PP_REMOVE_SPECIAL_CHARACTERS, PP_REMOVE_BASE_PUNCTUATIONS, PP_REMOVE_NUMBERS])
  if (len(text) > MAX_SEQUENCE_LENGTH):
    continue

  text = " ".join(text)

  texts.append(text)
  if "OFF" in line[2]:
    labels.append([1])
  else:
    labels.append([0])

print(str(count))
print("Loaded dataset")
print(str(len(texts)) + " texts")
print("First: " + texts[0])
print(labels[0])

20122
Loaded dataset
95742 texts
First: true or false the situation as of march was such a saudi proposal of land for peace and recognition by all arab countries was made the day the proposal was to be made formal by the arab league was the day the israeli is under the command of ariel sharon began the invasion of the palestinian selfrule areas user arab
[0]


In [148]:
import os
import numpy as np
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Prepare tokenizer given loaded texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
seq = tokenizer.texts_to_sequences(texts)

vocab_size = len(tokenizer.word_index)+1

# Load Glove model
EMBEDDING_DIM = 300
pad_seq = pad_sequences(seq,maxlen=MAX_SEQUENCE_LENGTH)
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# Create embedding matrix

word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Create embedding layer

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Created embedding layer

Found 400000 word vectors.


Train model

In [149]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional
from keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import F1Score


train_index = int(len(pad_seq) * 0.9)

train_seq = np.array(pad_seq[:train_index])
train_labels = np.array(labels[:train_index])

test_seq = np.array(pad_seq[train_index:])
test_labels = np.array(labels[train_index:])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

model = Sequential()
model.add(embedding_layer)
model.add(Dense(128,activation = 'relu'))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics = [F1Score(num_classes=1, threshold=0.5), 'accuracy'])

model.fit(train_seq, train_labels, epochs=10, validation_split=(1/6), batch_size=128, callbacks=[es])

Epoch 1/20
141/141 [==============================] - 38s 225ms/step - loss: 0.4602 - f1_score: 0.2720 - accuracy: 0.8170 - val_loss: 0.3288 - val_f1_score: 0.5172 - val_accuracy: 0.8700
Epoch 2/20
141/141 [==============================] - 30s 212ms/step - loss: 0.3539 - f1_score: 0.5558 - accuracy: 0.8623 - val_loss: 0.3305 - val_f1_score: 0.4955 - val_accuracy: 0.8703
Epoch 3/20
141/141 [==============================] - 30s 213ms/step - loss: 0.3441 - f1_score: 0.5482 - accuracy: 0.8667 - val_loss: 0.3240 - val_f1_score: 0.5494 - val_accuracy: 0.8737
Epoch 4/20
141/141 [==============================] - 30s 213ms/step - loss: 0.3379 - f1_score: 0.5880 - accuracy: 0.8706 - val_loss: 0.3183 - val_f1_score: 0.6032 - val_accuracy: 0.8729
Epoch 5/20
141/141 [==============================] - 30s 215ms/step - loss: 0.3277 - f1_score: 0.6003 - accuracy: 0.8733 - val_loss: 0.3123 - val_f1_score: 0.5738 - val_accuracy: 0.8766
Epoch 6/20
141/141 [==============================] - 30s 215ms/s

Evaluate model

In [150]:
# Evaluate
hm = model.evaluate(test_seq, test_labels, verbose=0, return_dict=True)
print(hm)
#print('Test accuracy:', hm['accuracy'])

TP=0
TN=0
FP=0
FN=0
total=0

predictions = (model.predict(test_seq) > 0.5).astype("int32")
for i in range(len(test_seq)):
  predicted_class = predictions[i][0]
  actual_class = test_labels[i][0]

  if actual_class == 1:
    if predicted_class == 1:
      TP += 1
    else:
      FN += 1
  else:
    if predicted_class == 0:
      TN += 1
    else:
      FP += 1
  
  total += 1

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)

print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1 Score: " + str(f1))
print("Accuracy: " + str(str((TP + TN) / total)))


{'loss': 0.42419368028640747, 'f1_score': array([0.5672534], dtype=float32), 'accuracy': 0.8639164566993713}
Precision: 0.6651090342679128
Recall: 0.4944991314418066
F1 Score: 0.5672534041846563
Accuracy: 0.8639164490861618


3
